In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# scikit-learn modules
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV

C:\Users\anilo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import tensorflow as tf
import os
import random as rn

# The below is necessary for starting Numpy generated random numbers in a well-defined initial state.
np.random.seed(42)

# The below is necessary for starting core Python generated random numbers in a well-defined state.
rn.seed(12345)

# The below set_seed() will make random number generation in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/random/set_seed
tf.random.set_seed(1234)


In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
def Evolution_Metrics(model,x_test,y_test):
    y_pred = model.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(color.BOLD)
    print('MAE: ',mae)
    print('MSE: ',mse)
    print('RMSE: ',rmse)
    print('R2: ',r2)
    print(color.END)

# Each RunId Bucket

In [4]:
def preprocess(df):
    SeaTemp='SW20'
    WindSpeed='WC0'
    Load='FAU'
    abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
    run_list = ['Time', 'Load', 'SeaTemp', 'WindSpeed', 'RunId']
    df_sample=df[abr_sensor_list+run_list]
    df_sample=df_sample[df_sample['SeaTemp']==SeaTemp]
    df_sample=df_sample[df_sample['WindSpeed']==WindSpeed]
    df_sample=df_sample[df_sample['Load']==Load]
    resample_list=[]
    for uniq in df_sample['RunId'].unique():
        s=df_sample[df_sample['RunId']==uniq]
        s.drop(columns=['Time', 'Load','SeaTemp','RunId','WindSpeed'],inplace=True)
        s.dropna(inplace=True)
        s.reset_index(drop=True,inplace=True)
        grouped_df = s.groupby(s.index // 4).mean()
        resample_list.append(grouped_df)
    
    df1 = pd.concat(resample_list, axis=0)
    df1.reset_index(drop=True,inplace=True)
    x = df1.drop('Z02013', axis = 1) # Features
    y = df1['Z02013']  # Target
    return x ,y

In [5]:
df=pd.read_csv(r'E:\Havelsan\Datasets\M0000_train_sensors.csv', engine='c')
df

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1410
1,00:00:01,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1409
2,00:00:02,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1408
3,00:00:03,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1407
4,00:00:04,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778855,00:23:47,FAU,SW28,WC6,35,12.4944,25.0092,3353.82,12.9058,2.91922,...,0.0,16.1273,6,16.1544,31.5922,0.0,1.0,0,0,4
778856,00:23:48,FAU,SW28,WC6,35,12.4761,24.9927,3353.82,12.7802,2.91487,...,0.0,16.1317,6,16.1275,31.5982,0.0,1.0,0,0,3
778857,00:23:49,FAU,SW28,WC6,35,12.4578,24.9762,3353.82,12.6545,2.91053,...,0.0,16.1361,6,16.1005,31.6042,0.0,1.0,0,0,2
778858,00:23:50,FAU,SW28,WC6,35,12.4395,24.9597,3353.82,12.5288,2.90618,...,0.0,16.1405,6,16.0736,31.6102,0.0,1.0,0,0,1


In [6]:
X_train , Y_train = preprocess(df)

C:\Users\anilo\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\anilo\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [7]:
X_train

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.033825,305.55825,56.30100,13.809175,2.639642,2.834907,2.640425,73.956575,7.789537,2.153763,...,43.533325,348.32400,305.32100,303.50975,-1.820360,43.796075,48.004975,220.36225,12.80620,12.864525
1,14.005075,327.46400,788.21375,13.775850,2.647335,2.836055,2.648525,73.934000,7.789335,2.154190,...,43.532350,348.46800,305.37300,303.56000,-1.828145,43.764700,48.004525,220.37500,12.80635,12.891500
2,14.042100,304.82850,1689.02750,13.818000,2.640332,2.837467,2.641055,73.962475,7.789135,2.154530,...,43.531150,348.41925,305.43700,303.61000,-1.837730,43.726050,48.003975,220.38750,12.80655,12.881025
3,14.079175,282.19250,2589.84500,13.860175,2.633325,2.838880,2.633585,73.990975,7.788935,2.154870,...,43.529950,348.37075,305.50200,303.66000,-1.847313,43.687400,48.003425,220.40050,12.80675,12.870575
4,14.101650,267.10325,3150.32500,13.885750,2.628635,2.839380,2.628602,74.008350,7.788915,2.155350,...,43.529200,348.34425,305.54825,303.70425,-1.846385,43.672975,48.003100,220.42000,12.80690,12.859875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11088,14.094350,261.61750,3131.57500,13.878475,2.625430,2.834192,2.625402,74.001700,7.789920,2.156467,...,43.535450,348.37600,305.46600,303.99700,-1.477685,43.990300,48.008700,220.40050,12.80430,12.822100
11089,14.096150,255.75850,3131.19000,13.880425,2.624820,2.833965,2.624785,74.001700,7.790100,2.156413,...,43.533500,348.37600,305.52100,304.06400,-1.468598,43.933500,48.008700,220.41100,12.80430,12.815700
11090,14.096925,250.91025,3130.95000,13.881100,2.624157,2.833580,2.624128,74.001700,7.790287,2.156290,...,43.532300,348.34000,305.55750,304.10950,-1.458375,43.887625,48.008700,220.42625,12.80430,12.808000
11091,14.096050,247.74675,3130.95000,13.879650,2.623415,2.832935,2.623400,74.001700,7.790495,2.156050,...,43.532300,348.24475,305.56225,304.11950,-1.446248,43.859750,48.008700,220.44850,12.80430,12.798200


In [8]:
Y_train

0        20.338525
1        20.543300
2        20.326950
3        20.110600
4        19.975850
           ...    
11088    19.969800
11089    19.976900
11090    19.981400
11091    19.981400
11092    19.981400
Name: Z02013, Length: 11093, dtype: float64

In [9]:
dft=pd.read_csv(r'E:\Havelsan\Datasets\M0000_test_sensors.csv', engine='c')
dft

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1413
1,00:00:01,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1412
2,00:00:02,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1411
3,00:00:03,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1410
4,00:00:04,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402488,00:19:20,FAU,SW28,WC6,53,13.5748,25.1889,3573.65,12.2076,3.11218,...,0.0,16.2952,6,16.9101,31.4515,0.0,1.0,0,0,4
402489,00:19:21,FAU,SW28,WC6,53,13.5252,25.1680,3573.65,11.9689,3.09877,...,0.0,16.2934,6,16.9109,31.3278,0.0,1.0,0,0,3
402490,00:19:22,FAU,SW28,WC6,53,13.4756,25.1472,3573.65,11.7302,3.08536,...,0.0,16.2917,6,16.9116,31.2041,0.0,1.0,0,0,2
402491,00:19:23,FAU,SW28,WC6,53,13.4259,25.1263,3573.65,11.4916,3.07195,...,0.0,16.2899,6,16.9123,31.0804,0.0,1.0,0,0,1


In [10]:
X_test , Y_test = preprocess(dft)

C:\Users\anilo\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\anilo\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [11]:
X_test

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.090400,266.67600,0.000,13.874100,2.626690,2.834820,2.626720,74.000500,7.789590,2.153590,...,43.5334,348.14300,305.31700,303.49700,-1.819760,43.798500,48.005000,220.35900,12.806200,12.829700
1,14.090400,266.67600,0.000,13.874100,2.626690,2.834820,2.626720,74.000500,7.789590,2.153590,...,43.5334,348.14300,305.31700,303.49700,-1.819760,43.798500,48.005000,220.35900,12.806200,12.829700
2,14.100750,281.11700,3150.940,13.884925,2.630625,2.840533,2.630565,74.000500,7.788330,2.153675,...,43.5301,348.54350,305.48425,303.63700,-1.847045,43.710650,48.004700,220.37050,12.806700,12.884375
3,14.100275,275.82300,3145.840,13.884425,2.629625,2.839863,2.629555,74.000500,7.788650,2.154358,...,43.5301,348.49950,305.51275,303.66850,-1.844360,43.702775,48.003925,220.39800,12.806700,12.870525
4,14.099850,270.52900,3140.740,13.883925,2.628625,2.839198,2.628550,74.000500,7.788963,2.155045,...,43.5301,348.45550,305.54150,303.70000,-1.841675,43.694875,48.003200,220.42500,12.806700,12.856700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5398,14.087250,287.06700,3133.060,13.871400,2.627970,2.834575,2.627970,73.997750,7.789360,2.148160,...,43.5261,348.33875,305.69850,304.29050,-1.405535,43.528975,48.010600,220.55900,12.805600,12.851350
5399,14.090325,283.65700,3134.010,13.874550,2.627970,2.835335,2.627970,73.999975,7.789360,2.148160,...,43.5261,348.29975,305.68850,304.27825,-1.404790,43.515125,48.010600,220.57575,12.804775,12.850925
5400,14.092850,279.38725,3134.625,13.877100,2.627740,2.835775,2.627740,74.001400,7.789405,2.148375,...,43.5263,348.24650,305.66675,304.25650,-1.403738,43.525850,48.010100,220.59000,12.804025,12.847725
5401,14.094550,273.68475,3134.685,13.878700,2.627130,2.835680,2.627123,74.001400,7.789518,2.148953,...,43.5269,348.17050,305.62550,304.21950,-1.402160,43.577450,48.008700,220.60000,12.803500,12.840000


In [12]:
Y_test

0       19.971500
1       19.971500
2       19.977875
3       19.979200
4       19.980525
          ...    
5398    20.023475
5399    20.006400
5400    19.996475
5401    19.998450
5402    20.000450
Name: Z02013, Length: 5403, dtype: float64

In [13]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
scaler=StandardScaler()
scaler.fit(Y_train.values.reshape(-1, 1))
Y_train_scaled = scaler.transform(Y_train.values.reshape(-1, 1))

# Basic

In [15]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [16]:
Evolution_Metrics(model_3,X_test,Y_test)


MAE:  0.0027729925820655674
MSE:  5.4642627382714834e-05
RMSE:  0.007392065163586887
R2:  0.9617558486457752



# Just X scaled

In [17]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [18]:
Evolution_Metrics(model_3,X_test_scaled,Y_test)


MAE:  0.0027747103123630375
MSE:  5.4742009389585e-05
RMSE:  0.007398784318358321
R2:  0.961686291585753



# X and Y Scaled

In [19]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train_scaled)

C:\Users\anilo\AppData\Local\Temp/ipykernel_17184/3914077697.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_3.fit(X_train_scaled, Y_train_scaled)


RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [20]:
# Make predictions
y_pred_scaled = model_3.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007359402548446988


# NT with X and Y scaled

In [21]:
model = Sequential()

model.add(Dense(96, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(384))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 96)                3456      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 96)                0         
                                                                 
 batch_normalization (BatchN  (None, 96)               384       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense_1 (Dense)             (None, 384)               37248     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 384)               0         
                                                        

In [22]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
278/278 [==============================] - 3s 5ms/step - loss: 0.6545 - mse: 0.6545 - val_loss: 0.2562 - val_mse: 0.2562
Epoch 2/100
278/278 [==============================] - 1s 5ms/step - loss: 0.3606 - mse: 0.3606 - val_loss: 0.2316 - val_mse: 0.2316
Epoch 3/100
278/278 [==============================] - 1s 4ms/step - loss: 0.2777 - mse: 0.2777 - val_loss: 0.1506 - val_mse: 0.1506
Epoch 4/100
278/278 [==============================] - 1s 5ms/step - loss: 0.2472 - mse: 0.2472 - val_loss: 0.2722 - val_mse: 0.2722
Epoch 5/100
278/278 [==============================] - 1s 4ms/step - loss: 0.2258 - mse: 0.2258 - val_loss: 0.1128 - val_mse: 0.1128
Epoch 6/100
278/278 [==============================] - 1s 4ms/step - loss: 0.2024 - mse: 0.2024 - val_loss: 0.1478 - val_mse: 0.1478
Epoch 7/100
278/278 [==============================] - 1s 5ms/step - loss: 0.2150 - mse: 0.2150 - val_loss: 0.1024 - val_mse: 0.1024
Epoch 8/100
278/278 [==============================] - 1s 5ms/step - 

In [23]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

169/169 [==============================] - 0s 2ms/step
Root Mean Squared Error: 0.00963894830361977


# Continuous RunId Bucket

In [24]:
def preprocess(df):
    SeaTemp='SW20'
    WindSpeed='WC0'
    Load='FAU'
    abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
    run_list = ['Time', 'Load', 'SeaTemp', 'WindSpeed', 'RunId']
    df_sample=df[abr_sensor_list+run_list]
    df_sample=df_sample[df_sample['SeaTemp']==SeaTemp]
    df_sample=df_sample[df_sample['WindSpeed']==WindSpeed]
    df_sample=df_sample[df_sample['Load']==Load]

    s=df_sample
    s.drop(columns=['Time', 'Load','SeaTemp','RunId','WindSpeed'],inplace=True)
    s.dropna(inplace=True)
    s.reset_index(drop=True,inplace=True)
    grouped_df = s.groupby(s.index // 4).mean()
    grouped_df.reset_index(drop=True,inplace=True)
    x = grouped_df.drop('Z02013', axis = 1) # Features
    y = grouped_df['Z02013']  # Target
    return x ,y

In [25]:
df=pd.read_csv(r'E:\Havelsan\Datasets\M0000_train_sensors.csv', engine='c')
df

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1410
1,00:00:01,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1409
2,00:00:02,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1408
3,00:00:03,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1407
4,00:00:04,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778855,00:23:47,FAU,SW28,WC6,35,12.4944,25.0092,3353.82,12.9058,2.91922,...,0.0,16.1273,6,16.1544,31.5922,0.0,1.0,0,0,4
778856,00:23:48,FAU,SW28,WC6,35,12.4761,24.9927,3353.82,12.7802,2.91487,...,0.0,16.1317,6,16.1275,31.5982,0.0,1.0,0,0,3
778857,00:23:49,FAU,SW28,WC6,35,12.4578,24.9762,3353.82,12.6545,2.91053,...,0.0,16.1361,6,16.1005,31.6042,0.0,1.0,0,0,2
778858,00:23:50,FAU,SW28,WC6,35,12.4395,24.9597,3353.82,12.5288,2.90618,...,0.0,16.1405,6,16.0736,31.6102,0.0,1.0,0,0,1


In [26]:
X_train , Y_train = preprocess(df)

In [27]:
X_train

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.033825,305.55825,56.30100,13.809175,2.639642,2.834907,2.640425,73.956575,7.789537,2.153763,...,43.533325,348.32400,305.32100,303.50975,-1.820360,43.796075,48.004975,220.36225,12.80620,12.864525
1,14.005075,327.46400,788.21375,13.775850,2.647335,2.836055,2.648525,73.934000,7.789335,2.154190,...,43.532350,348.46800,305.37300,303.56000,-1.828145,43.764700,48.004525,220.37500,12.80635,12.891500
2,14.042100,304.82850,1689.02750,13.818000,2.640332,2.837467,2.641055,73.962475,7.789135,2.154530,...,43.531150,348.41925,305.43700,303.61000,-1.837730,43.726050,48.003975,220.38750,12.80655,12.881025
3,14.079175,282.19250,2589.84500,13.860175,2.633325,2.838880,2.633585,73.990975,7.788935,2.154870,...,43.529950,348.37075,305.50200,303.66000,-1.847313,43.687400,48.003425,220.40050,12.80675,12.870575
4,14.101650,267.10325,3150.32500,13.885750,2.628635,2.839380,2.628602,74.008350,7.788915,2.155350,...,43.529200,348.34425,305.54825,303.70425,-1.846385,43.672975,48.003100,220.42000,12.80690,12.859875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,14.093450,264.54750,3131.76500,13.877525,2.625738,2.834305,2.625710,74.001700,7.789830,2.156492,...,43.536450,348.37600,305.43800,303.96375,-1.482225,44.018700,48.008700,220.39525,12.80430,12.825300
11078,14.095250,258.68800,3131.38250,13.879450,2.625125,2.834077,2.625095,74.001700,7.790010,2.156440,...,43.534450,348.37600,305.49350,304.03050,-1.473143,43.961900,48.008700,220.40575,12.80430,12.818900
11079,14.096875,252.99725,3131.02250,13.881175,2.624505,2.833825,2.624470,74.001700,7.790190,2.156375,...,43.532675,348.37000,305.54575,304.09375,-1.463865,43.906950,48.008700,220.41700,12.80430,12.812275
11080,14.096500,249.32850,3130.95000,13.880375,2.623785,2.833260,2.623762,74.001700,7.790393,2.156170,...,43.532300,348.29225,305.55975,304.11450,-1.452313,43.873675,48.008700,220.43750,12.80430,12.803100


In [28]:
Y_train

0        20.338525
1        20.543300
2        20.326950
3        20.110600
4        19.975850
           ...    
11077    19.966225
11078    19.973350
11079    19.980050
11080    19.981400
11081    19.981400
Name: Z02013, Length: 11082, dtype: float64

In [29]:
dft=pd.read_csv(r'E:\Havelsan\Datasets\M0000_test_sensors.csv', engine='c')
dft

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1413
1,00:00:01,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1412
2,00:00:02,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1411
3,00:00:03,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1410
4,00:00:04,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402488,00:19:20,FAU,SW28,WC6,53,13.5748,25.1889,3573.65,12.2076,3.11218,...,0.0,16.2952,6,16.9101,31.4515,0.0,1.0,0,0,4
402489,00:19:21,FAU,SW28,WC6,53,13.5252,25.1680,3573.65,11.9689,3.09877,...,0.0,16.2934,6,16.9109,31.3278,0.0,1.0,0,0,3
402490,00:19:22,FAU,SW28,WC6,53,13.4756,25.1472,3573.65,11.7302,3.08536,...,0.0,16.2917,6,16.9116,31.2041,0.0,1.0,0,0,2
402491,00:19:23,FAU,SW28,WC6,53,13.4259,25.1263,3573.65,11.4916,3.07195,...,0.0,16.2899,6,16.9123,31.0804,0.0,1.0,0,0,1


In [30]:
X_test , Y_test = preprocess(dft)

In [31]:
X_test

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.090400,266.67600,0.000,13.874100,2.626690,2.834820,2.626720,74.000500,7.789590,2.153590,...,43.533400,348.14300,305.31700,303.49700,-1.819760,43.798500,48.005000,220.35900,12.806200,12.829700
1,14.090400,266.67600,0.000,13.874100,2.626690,2.834820,2.626720,74.000500,7.789590,2.153590,...,43.533400,348.14300,305.31700,303.49700,-1.819760,43.798500,48.005000,220.35900,12.806200,12.829700
2,14.100750,281.11700,3150.940,13.884925,2.630625,2.840533,2.630565,74.000500,7.788330,2.153675,...,43.530100,348.54350,305.48425,303.63700,-1.847045,43.710650,48.004700,220.37050,12.806700,12.884375
3,14.100275,275.82300,3145.840,13.884425,2.629625,2.839863,2.629555,74.000500,7.788650,2.154358,...,43.530100,348.49950,305.51275,303.66850,-1.844360,43.702775,48.003925,220.39800,12.806700,12.870525
4,14.099850,270.52900,3140.740,13.883925,2.628625,2.839198,2.628550,74.000500,7.788963,2.155045,...,43.530100,348.45550,305.54150,303.70000,-1.841675,43.694875,48.003200,220.42500,12.806700,12.856700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5394,14.088775,285.36200,3133.535,13.873000,2.627970,2.834955,2.627970,73.998850,7.789360,2.148160,...,43.526100,348.31925,305.69350,304.28450,-1.405162,43.522050,48.010600,220.56725,12.805200,12.851150
5395,14.091750,281.80875,3134.430,13.876000,2.627933,2.835662,2.627933,74.000975,7.789367,2.148195,...,43.526125,348.27775,305.68150,304.27025,-1.404367,43.512300,48.010525,220.58375,12.804350,12.850225
5396,14.093700,276.53600,3134.655,13.877900,2.627435,2.835727,2.627430,74.001400,7.789462,2.148665,...,43.526600,348.20850,305.64625,304.23800,-1.402950,43.551650,48.009400,220.59500,12.803775,12.843875
5397,14.095400,270.83350,3134.715,13.879500,2.626823,2.835633,2.626815,74.001400,7.789573,2.149240,...,43.527200,348.13250,305.60475,304.20100,-1.401372,43.603250,48.008025,220.60500,12.803200,12.836150


In [32]:
Y_test

0       19.971500
1       19.971500
2       19.977875
3       19.979200
4       19.980525
          ...    
5394    20.014925
5395    19.999075
5396    19.997450
5397    19.999450
5398    20.000950
Name: Z02013, Length: 5399, dtype: float64

In [33]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [34]:
scaler=StandardScaler()
scaler.fit(Y_train.values.reshape(-1, 1))
Y_train_scaled = scaler.transform(Y_train.values.reshape(-1, 1))

# BASİC

In [35]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [36]:
Evolution_Metrics(model_3,X_test,Y_test)


MAE:  0.0027987599816744046
MSE:  5.79559456482788e-05
RMSE:  0.007612880246547873
R2:  0.9594304071008162



# Just X Scaled

In [37]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [38]:
Evolution_Metrics(model_3,X_test_scaled,Y_test)


MAE:  0.002799503598677539
MSE:  5.804226747386257e-05
RMSE:  0.007618547596088284
R2:  0.9593699811810408



#  X and Y scaled

In [39]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train_scaled)

C:\Users\anilo\AppData\Local\Temp/ipykernel_17184/3914077697.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_3.fit(X_train_scaled, Y_train_scaled)


RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [40]:
# Make predictions
y_pred_scaled = model_3.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.007672439602829443


#  Nt with X and Y scaled

In [41]:
model = Sequential()

model.add(Dense(96, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(384))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 96)                3456      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 96)                0         
                                                                 
 batch_normalization_3 (Batc  (None, 96)               384       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 96)                0         
                                                                 
 dense_5 (Dense)             (None, 384)               37248     
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 384)               0         
                                                      

In [42]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
278/278 [==============================] - 3s 6ms/step - loss: 0.5778 - mse: 0.5778 - val_loss: 0.3711 - val_mse: 0.3711
Epoch 2/100
278/278 [==============================] - 1s 5ms/step - loss: 0.3234 - mse: 0.3234 - val_loss: 0.4734 - val_mse: 0.4734
Epoch 3/100
278/278 [==============================] - 1s 5ms/step - loss: 0.2722 - mse: 0.2722 - val_loss: 0.9015 - val_mse: 0.9015
Epoch 4/100
278/278 [==============================] - 1s 5ms/step - loss: 0.2401 - mse: 0.2401 - val_loss: 0.2433 - val_mse: 0.2433
Epoch 5/100
278/278 [==============================] - 1s 5ms/step - loss: 0.2243 - mse: 0.2243 - val_loss: 0.1547 - val_mse: 0.1547
Epoch 6/100
278/278 [==============================] - 1s 5ms/step - loss: 0.1960 - mse: 0.1960 - val_loss: 0.1903 - val_mse: 0.1903
Epoch 7/100
278/278 [==============================] - 1s 5ms/step - loss: 0.1875 - mse: 0.1875 - val_loss: 0.0841 - val_mse: 0.0841
Epoch 8/100
278/278 [==============================] - 1s 5ms/step - 

In [43]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

169/169 [==============================] - 0s 2ms/step
Root Mean Squared Error: 0.008941547084876154


# Each RunId Window


In [44]:
def preprocess(df):
    SeaTemp='SW20'
    WindSpeed='WC0'
    Load='FAU'
    abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
    run_list = ['Time', 'Load', 'SeaTemp', 'WindSpeed', 'RunId']
    df_sample=df[abr_sensor_list+run_list]
    df_sample=df_sample[df_sample['SeaTemp']==SeaTemp]
    df_sample=df_sample[df_sample['WindSpeed']==WindSpeed]
    df_sample=df_sample[df_sample['Load']==Load]
    resample_list=[]
    for uniq in df_sample['RunId'].unique():
        s=df_sample[df_sample['RunId']==uniq]
        s.drop(columns=['Time', 'Load','SeaTemp','RunId','WindSpeed'],inplace=True)
        s.dropna(inplace=True)
        s.reset_index(drop=True,inplace=True)
        rolling_mean = s.rolling(window=4).mean()
        rolling_mean.dropna(inplace=True)
        rolling_mean.reset_index(drop=True,inplace=True)
        resample_list.append(rolling_mean)
    df1 = pd.concat(resample_list, axis=0)
    df1.reset_index(drop=True,inplace=True)
    x = df1.drop('Z02013', axis = 1) # Features
    y = df1['Z02013']  # Target
    return x ,y

In [45]:
df=pd.read_csv(r'E:\Havelsan\Datasets\M0000_train_sensors.csv', engine='c')
df

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1410
1,00:00:01,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1409
2,00:00:02,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1408
3,00:00:03,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1407
4,00:00:04,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778855,00:23:47,FAU,SW28,WC6,35,12.4944,25.0092,3353.82,12.9058,2.91922,...,0.0,16.1273,6,16.1544,31.5922,0.0,1.0,0,0,4
778856,00:23:48,FAU,SW28,WC6,35,12.4761,24.9927,3353.82,12.7802,2.91487,...,0.0,16.1317,6,16.1275,31.5982,0.0,1.0,0,0,3
778857,00:23:49,FAU,SW28,WC6,35,12.4578,24.9762,3353.82,12.6545,2.91053,...,0.0,16.1361,6,16.1005,31.6042,0.0,1.0,0,0,2
778858,00:23:50,FAU,SW28,WC6,35,12.4395,24.9597,3353.82,12.5288,2.90618,...,0.0,16.1405,6,16.0736,31.6102,0.0,1.0,0,0,1


In [46]:
X_train , Y_train = preprocess(df)

C:\Users\anilo\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\anilo\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [47]:
X_train

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.033825,305.55825,56.30100,13.809175,2.639642,2.834907,2.640425,73.956575,7.789537,2.153763,...,43.533325,348.32400,305.32100,303.50975,-1.820360,43.796075,48.004975,220.36225,12.806200,12.864525
1,14.009025,322.87725,168.90300,13.780650,2.645460,2.835085,2.646578,73.937275,7.789492,2.153880,...,43.533175,348.40975,305.32900,303.52075,-1.821557,43.791250,48.004900,220.36500,12.806225,12.880950
2,13.986525,338.78150,337.80600,13.754775,2.650840,2.835350,2.652262,73.919750,7.789435,2.154020,...,43.532950,348.49250,305.34100,303.53500,-1.823355,43.784000,48.004800,220.36850,12.806250,12.896725
3,13.995800,333.12275,563.00975,13.765300,2.649087,2.835703,2.650395,73.926875,7.789385,2.154105,...,43.532650,348.48025,305.35700,303.54750,-1.825750,43.774350,48.004675,220.37175,12.806300,12.894100
4,14.005075,327.46400,788.21375,13.775850,2.647335,2.836055,2.648525,73.934000,7.789335,2.154190,...,43.532350,348.46800,305.37300,303.56000,-1.828145,43.764700,48.004525,220.37500,12.806350,12.891500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44218,14.096500,249.32850,3130.95000,13.880375,2.623785,2.833260,2.623762,74.001700,7.790392,2.156170,...,43.532300,348.29225,305.55975,304.11450,-1.452313,43.873675,48.008700,220.43750,12.804300,12.803100
44219,14.096275,248.53750,3130.95000,13.880025,2.623600,2.833098,2.623580,74.001700,7.790445,2.156110,...,43.532300,348.26850,305.56100,304.11700,-1.449280,43.866725,48.008700,220.44300,12.804300,12.800650
44220,14.096050,247.74675,3130.95000,13.879650,2.623415,2.832935,2.623400,74.001700,7.790495,2.156050,...,43.532300,348.24475,305.56225,304.11950,-1.446248,43.859750,48.008700,220.44850,12.804300,12.798200
44221,14.095825,246.95600,3130.95000,13.879300,2.623230,2.832773,2.623220,74.001700,7.790545,2.155990,...,43.532300,348.22100,305.56350,304.12200,-1.443215,43.852775,48.008700,220.45425,12.804300,12.795750


In [48]:
Y_train

0        20.338525
1        20.501750
2        20.651450
3        20.597375
4        20.543300
           ...    
44218    19.981400
44219    19.981400
44220    19.981400
44221    19.981400
44222    19.981400
Name: Z02013, Length: 44223, dtype: float64

In [49]:
dft=pd.read_csv(r'E:\Havelsan\Datasets\M0000_test_sensors.csv', engine='c')
dft

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1413
1,00:00:01,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1412
2,00:00:02,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1411
3,00:00:03,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1410
4,00:00:04,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402488,00:19:20,FAU,SW28,WC6,53,13.5748,25.1889,3573.65,12.2076,3.11218,...,0.0,16.2952,6,16.9101,31.4515,0.0,1.0,0,0,4
402489,00:19:21,FAU,SW28,WC6,53,13.5252,25.1680,3573.65,11.9689,3.09877,...,0.0,16.2934,6,16.9109,31.3278,0.0,1.0,0,0,3
402490,00:19:22,FAU,SW28,WC6,53,13.4756,25.1472,3573.65,11.7302,3.08536,...,0.0,16.2917,6,16.9116,31.2041,0.0,1.0,0,0,2
402491,00:19:23,FAU,SW28,WC6,53,13.4259,25.1263,3573.65,11.4916,3.07195,...,0.0,16.2899,6,16.9123,31.0804,0.0,1.0,0,0,1


In [50]:
X_test , Y_test = preprocess(dft)

C:\Users\anilo\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\anilo\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [51]:
X_test

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
1,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
2,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
3,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
4,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21535,14.094550,273.68475,3134.685,13.8787,2.627130,2.835680,2.627123,74.0014,7.789518,2.148953,...,43.52690,348.1705,305.62550,304.21950,-1.402160,43.57745,48.008700,220.6000,12.803500,12.840000
21536,14.094975,272.25925,3134.700,13.8791,2.626977,2.835655,2.626970,74.0014,7.789545,2.149095,...,43.52705,348.1515,305.61525,304.21025,-1.401768,43.59035,48.008375,220.6025,12.803350,12.838075
21537,14.095400,270.83350,3134.715,13.8795,2.626823,2.835633,2.626815,74.0014,7.789572,2.149240,...,43.52720,348.1325,305.60475,304.20100,-1.401372,43.60325,48.008025,220.6050,12.803200,12.836150
21538,14.095800,269.40775,3134.730,13.8799,2.626670,2.835608,2.626660,74.0014,7.789600,2.149385,...,43.52735,348.1135,305.59450,304.19175,-1.400980,43.61615,48.007700,220.6075,12.803075,12.834225


In [52]:
Y_test

0        19.97150
1        19.97150
2        19.97150
3        19.97150
4        19.97150
           ...   
21535    19.99845
21536    19.99895
21537    19.99945
21538    19.99995
21539    20.00045
Name: Z02013, Length: 21540, dtype: float64

In [53]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [54]:
scaler=StandardScaler()
scaler.fit(Y_train.values.reshape(-1, 1))
Y_train_scaled = scaler.transform(Y_train.values.reshape(-1, 1))

# Basic

In [55]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [56]:
Evolution_Metrics(model_3,X_test,Y_test)


MAE:  0.00175818950284898
MSE:  2.883931407245157e-05
RMSE:  0.005370224769267257
R2:  0.9793399600733341



# Just X Scaled

In [57]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [58]:
Evolution_Metrics(model_3,X_test_scaled,Y_test)


MAE:  0.0017567881059813077
MSE:  2.8575082645209002e-05
RMSE:  0.005345566634624341
R2:  0.979529251393613



# X and Y scaled

In [59]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train_scaled)

C:\Users\anilo\AppData\Local\Temp/ipykernel_17184/3914077697.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_3.fit(X_train_scaled, Y_train_scaled)


RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [60]:
# Make predictions
y_pred_scaled = model_3.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.005356456912889458


# Nt with X and Y Scaled

In [61]:
model = Sequential()

model.add(Dense(96, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(384))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 96)                3456      
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 96)                0         
                                                                 
 batch_normalization_6 (Batc  (None, 96)               384       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 96)                0         
                                                                 
 dense_9 (Dense)             (None, 384)               37248     
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 384)               0         
                                                      

In [62]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1106/1106 [==============================] - 8s 5ms/step - loss: 0.3383 - mse: 0.3383 - val_loss: 0.1811 - val_mse: 0.1811
Epoch 2/100
1106/1106 [==============================] - 6s 6ms/step - loss: 0.1878 - mse: 0.1878 - val_loss: 0.1194 - val_mse: 0.1194
Epoch 3/100
1106/1106 [==============================] - 6s 5ms/step - loss: 0.1628 - mse: 0.1628 - val_loss: 0.1045 - val_mse: 0.1045
Epoch 4/100
1106/1106 [==============================] - 6s 5ms/step - loss: 0.1354 - mse: 0.1354 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 5/100
1106/1106 [==============================] - 6s 5ms/step - loss: 0.1290 - mse: 0.1290 - val_loss: 0.0904 - val_mse: 0.0904
Epoch 6/100
1106/1106 [==============================] - 6s 5ms/step - loss: 0.1199 - mse: 0.1199 - val_loss: 0.0564 - val_mse: 0.0564
Epoch 7/100
1106/1106 [==============================] - 6s 5ms/step - loss: 0.1079 - mse: 0.1079 - val_loss: 0.1132 - val_mse: 0.1132
Epoch 8/100
1106/1106 [==============================] 

In [63]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

674/674 [==============================] - 1s 2ms/step
Root Mean Squared Error: 0.007668880654953586


#  Continuous RunId Window

In [67]:
def preprocess(df):
    SeaTemp='SW20'
    WindSpeed='WC0'
    Load='FAU'
    abr_sensor_list = ['E02005', 'E02006', 'E02056', 'E03760', 'G00027', 'G00108', 'G02011', 'N02015', 'P00023', 'P01005', 'P01302', 'P01303', 'P01600', 'P01602', 'P02055', 'P02065', 'P02066', 'P02071', 'P02072', 'Q02004', 'T00002', 'T01010', 'T01011', 'T01350', 'T01351', 'T01601', 'T01603', 'T02014', 'T02040', 'T02041', 'T02042', 'T02044', 'T04600', 'Z00518', 'Z01970', 'Z02013']
    run_list = ['Time', 'Load', 'SeaTemp', 'WindSpeed', 'RunId']
    df_sample=df[abr_sensor_list+run_list]
    df_sample=df_sample[df_sample['SeaTemp']==SeaTemp]
    df_sample=df_sample[df_sample['WindSpeed']==WindSpeed]
    df_sample=df_sample[df_sample['Load']==Load]

    s=df_sample
    s.drop(columns=['Time', 'Load','SeaTemp','RunId','WindSpeed'],inplace=True)
    s.dropna(inplace=True)
    s.reset_index(drop=True,inplace=True)
    rolling_mean = s.rolling(window=4).mean()
    rolling_mean.dropna(inplace=True)
    rolling_mean.reset_index(drop=True,inplace=True)


    x = rolling_mean.drop('Z02013', axis = 1) # Features
    y = rolling_mean['Z02013']  # Target
    return x ,y

In [68]:
df=pd.read_csv(r'E:\Havelsan\Datasets\M0000_train_sensors.csv', engine='c')
df

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1410
1,00:00:01,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1409
2,00:00:02,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1408
3,00:00:03,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1407
4,00:00:04,FAL,SW20,WC0,1,16.7659,430.9120,3647.98,16.5258,3.17649,...,0.0,12.9311,0,14.9813,22.5320,0.0,0.0,0,0,1406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778855,00:23:47,FAU,SW28,WC6,35,12.4944,25.0092,3353.82,12.9058,2.91922,...,0.0,16.1273,6,16.1544,31.5922,0.0,1.0,0,0,4
778856,00:23:48,FAU,SW28,WC6,35,12.4761,24.9927,3353.82,12.7802,2.91487,...,0.0,16.1317,6,16.1275,31.5982,0.0,1.0,0,0,3
778857,00:23:49,FAU,SW28,WC6,35,12.4578,24.9762,3353.82,12.6545,2.91053,...,0.0,16.1361,6,16.1005,31.6042,0.0,1.0,0,0,2
778858,00:23:50,FAU,SW28,WC6,35,12.4395,24.9597,3353.82,12.5288,2.90618,...,0.0,16.1405,6,16.0736,31.6102,0.0,1.0,0,0,1


In [69]:
X_train , Y_train = preprocess(df)

In [70]:
X_train

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.033825,305.55825,56.30100,13.809175,2.639642,2.834907,2.640425,73.956575,7.789537,2.153763,...,43.533325,348.32400,305.32100,303.50975,-1.820360,43.796075,48.004975,220.36225,12.806200,12.864525
1,14.009025,322.87725,168.90300,13.780650,2.645460,2.835085,2.646578,73.937275,7.789492,2.153880,...,43.533175,348.40975,305.32900,303.52075,-1.821557,43.791250,48.004900,220.36500,12.806225,12.880950
2,13.986525,338.78150,337.80600,13.754775,2.650840,2.835350,2.652262,73.919750,7.789435,2.154020,...,43.532950,348.49250,305.34100,303.53500,-1.823355,43.784000,48.004800,220.36850,12.806250,12.896725
3,13.995800,333.12275,563.00975,13.765300,2.649087,2.835703,2.650395,73.926875,7.789385,2.154105,...,43.532650,348.48025,305.35700,303.54750,-1.825750,43.774350,48.004675,220.37175,12.806300,12.894100
4,14.005075,327.46400,788.21375,13.775850,2.647335,2.836055,2.648525,73.934000,7.789335,2.154190,...,43.532350,348.46800,305.37300,303.56000,-1.828145,43.764700,48.004525,220.37500,12.806350,12.891500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44320,14.096500,249.32850,3130.95000,13.880375,2.623785,2.833260,2.623762,74.001700,7.790393,2.156170,...,43.532300,348.29225,305.55975,304.11450,-1.452313,43.873675,48.008700,220.43750,12.804300,12.803100
44321,14.096275,248.53750,3130.95000,13.880025,2.623600,2.833098,2.623580,74.001700,7.790445,2.156110,...,43.532300,348.26850,305.56100,304.11700,-1.449280,43.866725,48.008700,220.44300,12.804300,12.800650
44322,14.096050,247.74675,3130.95000,13.879650,2.623415,2.832935,2.623400,74.001700,7.790495,2.156050,...,43.532300,348.24475,305.56225,304.11950,-1.446248,43.859750,48.008700,220.44850,12.804300,12.798200
44323,14.095825,246.95600,3130.95000,13.879300,2.623230,2.832772,2.623220,74.001700,7.790545,2.155990,...,43.532300,348.22100,305.56350,304.12200,-1.443215,43.852775,48.008700,220.45425,12.804300,12.795750


In [71]:
Y_train

0        20.338525
1        20.501750
2        20.651450
3        20.597375
4        20.543300
           ...    
44320    19.981400
44321    19.981400
44322    19.981400
44323    19.981400
44324    19.981400
Name: Z02013, Length: 44325, dtype: float64

In [72]:
dft=pd.read_csv(r'E:\Havelsan\Datasets\M0000_test_sensors.csv', engine='c')
dft

,Time,Load,SeaTemp,WindSpeed,RunId,E02005,E02006,E02056,E03760,G00027,...,X02445,Z00518,Z00770,Z01970,Z02013,Z02436,Z02437,Z02477,Z02482,Linear
0,00:00:00,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1413
1,00:00:01,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1412
2,00:00:02,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1411
3,00:00:03,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1410
4,00:00:04,FAL,SW20,WC0,36,16.7659,430.9120,3647.98,NaN,3.17649,...,NaN,NaN,0,14.9813,22.5320,0.0,0.0,0,0,1409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402488,00:19:20,FAU,SW28,WC6,53,13.5748,25.1889,3573.65,12.2076,3.11218,...,0.0,16.2952,6,16.9101,31.4515,0.0,1.0,0,0,4
402489,00:19:21,FAU,SW28,WC6,53,13.5252,25.1680,3573.65,11.9689,3.09877,...,0.0,16.2934,6,16.9109,31.3278,0.0,1.0,0,0,3
402490,00:19:22,FAU,SW28,WC6,53,13.4756,25.1472,3573.65,11.7302,3.08536,...,0.0,16.2917,6,16.9116,31.2041,0.0,1.0,0,0,2
402491,00:19:23,FAU,SW28,WC6,53,13.4259,25.1263,3573.65,11.4916,3.07195,...,0.0,16.2899,6,16.9123,31.0804,0.0,1.0,0,0,1


In [73]:
X_test , Y_test = preprocess(dft)

In [74]:
X_test

,E02005,E02006,E02056,E03760,G00027,G00108,G02011,N02015,P00023,P01005,...,T01601,T01603,T02014,T02040,T02041,T02042,T02044,T04600,Z00518,Z01970
0,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
1,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
2,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
3,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
4,14.090400,266.67600,0.000,13.8741,2.626690,2.834820,2.626720,74.0005,7.789590,2.153590,...,43.53340,348.1430,305.31700,303.49700,-1.819760,43.79850,48.005000,220.3590,12.806200,12.829700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21586,14.094550,273.68475,3134.685,13.8787,2.627130,2.835680,2.627123,74.0014,7.789517,2.148953,...,43.52690,348.1705,305.62550,304.21950,-1.402160,43.57745,48.008700,220.6000,12.803500,12.840000
21587,14.094975,272.25925,3134.700,13.8791,2.626978,2.835655,2.626970,74.0014,7.789545,2.149095,...,43.52705,348.1515,305.61525,304.21025,-1.401768,43.59035,48.008375,220.6025,12.803350,12.838075
21588,14.095400,270.83350,3134.715,13.8795,2.626823,2.835633,2.626815,74.0014,7.789573,2.149240,...,43.52720,348.1325,305.60475,304.20100,-1.401372,43.60325,48.008025,220.6050,12.803200,12.836150
21589,14.095800,269.40775,3134.730,13.8799,2.626670,2.835608,2.626660,74.0014,7.789600,2.149385,...,43.52735,348.1135,305.59450,304.19175,-1.400980,43.61615,48.007700,220.6075,12.803075,12.834225


In [75]:
Y_test

0        19.97150
1        19.97150
2        19.97150
3        19.97150
4        19.97150
           ...   
21586    19.99845
21587    19.99895
21588    19.99945
21589    19.99995
21590    20.00045
Name: Z02013, Length: 21591, dtype: float64

In [76]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [77]:
scaler=StandardScaler()
scaler.fit(Y_train.values.reshape(-1, 1))
Y_train_scaled = scaler.transform(Y_train.values.reshape(-1, 1))

# Basic

In [78]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [79]:
Evolution_Metrics(model_3,X_test,Y_test)


MAE:  0.0018230251534475423
MSE:  3.451265109761908e-05
RMSE:  0.005874746896472995
R2:  0.9756628022987356



# X Scaled

In [80]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train)

RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [81]:
Evolution_Metrics(model_3,X_test_scaled,Y_test)


MAE:  0.0018275384562798872
MSE:  3.522819651537803e-05
RMSE:  0.005935334574847321
R2:  0.9751582229707965



#  X and Y Scaled

In [82]:
# Create a RandomForestRegressor with the best parameters
model_3 = RandomForestRegressor(
   n_estimators= 400, 
    min_samples_split=5, 
    min_samples_leaf= 1, 
    max_features="log2", 
    max_depth=30, 
    bootstrap=False,
    random_state=42,  # Seed
    n_jobs=-1  # Use all processors
)

model_3.fit(X_train_scaled, Y_train_scaled)

C:\Users\anilo\AppData\Local\Temp/ipykernel_17184/3914077697.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_3.fit(X_train_scaled, Y_train_scaled)


RandomForestRegressor(bootstrap=False, max_depth=30, max_features='log2',
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42)

In [83]:
# Make predictions
y_pred_scaled = model_3.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 0.005856003273663112


# NT with X and Y Scaled

In [84]:
model = Sequential()

model.add(Dense(96, input_dim=X_train_scaled.shape[1]))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(384))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.30000000000000004))

model.add(Dense(32))
model.add(LeakyReLU())
model.add(BatchNormalization())

model.add(Dense(1))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Define learning rate decay
lr_schedule = ExponentialDecay(initial_learning_rate=0.001,
                               decay_steps=10000,
                               decay_rate=0.9)

optimizer = Adam(learning_rate=lr_schedule)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 96)                3456      
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 96)                0         
                                                                 
 batch_normalization_9 (Batc  (None, 96)               384       
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 96)                0         
                                                                 
 dense_13 (Dense)            (None, 384)               37248     
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 384)               0         
                                                      

In [85]:
history = model.fit(X_train_scaled, Y_train_scaled, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

Epoch 1/100
1109/1109 [==============================] - 9s 6ms/step - loss: 0.3364 - mse: 0.3364 - val_loss: 0.1680 - val_mse: 0.1680
Epoch 2/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1873 - mse: 0.1873 - val_loss: 0.1980 - val_mse: 0.1980
Epoch 3/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1564 - mse: 0.1564 - val_loss: 0.0926 - val_mse: 0.0926
Epoch 4/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1361 - mse: 0.1361 - val_loss: 0.0805 - val_mse: 0.0805
Epoch 5/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1299 - mse: 0.1299 - val_loss: 0.0805 - val_mse: 0.0805
Epoch 6/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1275 - mse: 0.1275 - val_loss: 0.0716 - val_mse: 0.0716
Epoch 7/100
1109/1109 [==============================] - 7s 6ms/step - loss: 0.1190 - mse: 0.1190 - val_loss: 0.0582 - val_mse: 0.0582
Epoch 8/100
1109/1109 [==============================] 

In [86]:
# Make predictions
y_pred_scaled = model.predict(X_test_scaled)

# Inverse transform the predicted values
y_pred = scaler.inverse_transform(y_pred_scaled)
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

675/675 [==============================] - 1s 2ms/step
Root Mean Squared Error: 0.008452254554433804
